In [124]:
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from lib.residental_data_source import ResidentialDataSource
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [125]:
data_source = ResidentialDataSource('src/dataset_prepared.xlsx')
ds = data_source.fetch_data_set()

## Standardization



### int, float

In [126]:
def standardize_numeric_data(ds, target_column='Таргет'):

    numerical_data = ds.select_dtypes(include=['int64', 'float64']).drop(columns=[target_column])

    std = StandardScaler()
    std_data = std.fit_transform(numerical_data)
    cols = std.get_feature_names_out()
    
    std_ds = pd.DataFrame(std_data, columns=cols)
    ds = pd.concat([ds.drop(columns=cols), std_ds], axis=1)
    
    return ds


In [127]:
ds = standardize_numeric_data(ds, target_column='Таргет')
ds.head()

,Населенный пункт,Группа компаний,Класс недвижимости,Статус,Месяц,Таргет,Дата получения РС,Дата публикации ИНС,Всероссийский застройщик,Район,...,Расстояние по навигатору до: Луговая,Расстояние по навигатору до: Центр,Расстояние по навигатору до: Первая речка,Расстояние по навигатору до: Вторая речка,Расстояние по навигатору до: Третья рабочая,Расстояние по навигатору до: Окатовая,Расстояние по навигатору до: Баляева,Расстояние по навигатору до: Кампус ДВФУ,Расстояние по навигатору до: Артем,Расстояние по навигатору до: В-надеждинское
0,Артем,СЗВОСХОД,Типовой,Строится,Апрель 2021,0.0,19.08.2020,16.04.2021,Нет,Артем,...,3.510078,3.677804,3.473117,3.476863,3.673838,3.443989,3.585868,3.077331,-3.081237,-1.643038
1,Артем,СЗВОСХОД,Типовой,Строится,Май 2021,0.0,19.08.2020,16.04.2021,Нет,Артем,...,3.510078,3.677804,3.473117,3.476863,3.673838,3.443989,3.585868,3.077331,-3.081237,-1.643038
2,Артем,СЗВОСХОД,Типовой,Строится,Июнь 2021,0.0,19.08.2020,16.04.2021,Нет,Артем,...,3.510078,3.677804,3.473117,3.476863,3.673838,3.443989,3.585868,3.077331,-3.081237,-1.643038
3,Артем,СЗВОСХОД,Типовой,Строится,Июль 2021,0.0,19.08.2020,16.04.2021,Нет,Артем,...,3.510078,3.677804,3.473117,3.476863,3.673838,3.443989,3.585868,3.077331,-3.081237,-1.643038
4,Артем,СЗВОСХОД,Типовой,Строится,Август 2021,95000.0,19.08.2020,16.04.2021,Нет,Артем,...,3.510078,3.677804,3.473117,3.476863,3.673838,3.443989,3.585868,3.077331,-3.081237,-1.643038


### object


In [128]:
ids = (ds.dtypes == object)
object_columns = ds.columns[ids]
object_columns

Index(['Населенный пункт', 'Группа компаний', 'Класс недвижимости', 'Статус',
       'Месяц', 'Дата получения РС', 'Дата публикации ИНС',
       'Всероссийский застройщик', 'Район', 'Инфраструктура: Рядом ВУЗ',
       'Есть железная дорога 250м', 'Есть ЛЭП 100м',
       'Есть магистральные дороги 100м', 'Есть АЗС 100м', 'Есть скверы 500м',
       'Есть лес 500м', 'Есть водоем 500м', 'Есть пляж 1000м',
       'Есть тюрьма 250м', 'Есть ПНД 250м', 'Есть промзона 250м',
       'Есть гаражи 500м', 'Станции электропоезда 1000м',
       'Уехать без пересадок: Луговая', 'Уехать без пересадок: Центр',
       'Уехать без пересадок: Первая речка',
       'Уехать без пересадок: Вторая речка',
       'Уехать без пересадок: Третья рабочая',
       'Уехать без пересадок: Окатовая', 'Уехать без пересадок: Баляева',
       'Уехать без пересадок: Кампус ДВФУ', 'Основной материал фасада',
       'Вспомогательный материал фасада', 'Тип дома', 'Тип фасада',
       'Есть квартиры без отделки', 'Есть квартир

### Обработка месяца

In [129]:
ds['Месяц']

0       Апрель 2021
1          Май 2021
2         Июнь 2021
3         Июль 2021
4       Август 2021
           ...     
1602    Апрель 2022
1603       Май 2022
1604      Июнь 2022
1605      Июль 2022
1606    Август 2022
Name: Месяц, Length: 1607, dtype: object

In [130]:
months_mapping = {
    'январь': '1', 'февраль': '2', 'март': '3', 'апрель': '4', 'май': '5', 'июнь': '6',
    'июль': '7', 'август': '8', 'сентябрь': '9', 'октябрь': '10', 'ноябрь': '11', 'декабрь': '12'
}

month = ds['Месяц'].str.split().str[0].str.lower()
ds['Месяц'] = month.map(months_mapping)
ds['Месяц']

0       4
1       5
2       6
3       7
4       8
       ..
1602    4
1603    5
1604    6
1605    7
1606    8
Name: Месяц, Length: 1607, dtype: object

Обработка регистра и уникальных значений

In [131]:
object_list = [
    'Инфраструктура: Рядом ВУЗ', 
    'Есть железная дорога 250м',
    'Есть ЛЭП 100м',
    'Есть магистральные дороги 100м',
    'Есть АЗС 100м',
    'Есть скверы 500м',
    'Есть лес 500м',
    'Есть водоем 500м',
    'Есть пляж 1000м',
    'Есть тюрьма 250м',
    'Есть ПНД 250м',
    'Есть промзона 250м',
    'Есть гаражи 500м',
    'Есть велопарковка',
    'Есть квартиры с чистовой отделкой',
    'Есть площадки для собак', 
    'Есть квартиры под ключ', 
    'Есть квартиры без отделки', 
    'Есть квартиры с черновой отделкой', 
    'Уехать без пересадок: Луговая',
    'Уехать без пересадок: Центр',
    'Уехать без пересадок: Первая речка',
    'Уехать без пересадок: Вторая речка',
    'Уехать без пересадок: Третья рабочая',
    'Уехать без пересадок: Окатовая',
    'Уехать без пересадок: Баляева',
    'Уехать без пересадок: Кампус ДВФУ',
    'Внутренний двор без машин', 
    'Видеонаблюдение во дворе', 
    'Закрытая территория', 
    'Инфраструктура: Рядом ВУЗ', 
    'Всероссийский застройщик', 
    'Станции электропоезда 1000м']

In [132]:
for col in object_columns:
    try:
        ds[col] = ds[col].str.lower().str.strip()
    except Exception as e:
        print(f"Ошибка в столбце '{col}': {e}")

In [133]:
for col in object_list:
    num_unique = len(ds[col].unique())
    if num_unique != 2:
        print(f"Column '{col}' has {num_unique} unique values: {ds[col].unique()}")

Column 'Есть велопарковка' has 3 unique values: ['да' 'нет' 'неизвестно']
Column 'Есть квартиры с чистовой отделкой' has 3 unique values: ['да' 'нет' 'неизвестно']
Column 'Есть площадки для собак' has 3 unique values: ['нет' 'неизвестно' 'да']
Column 'Есть квартиры под ключ' has 3 unique values: ['да' 'нет' 'неизвестно']
Column 'Есть квартиры без отделки' has 3 unique values: ['да' 'нет' 'неизвестно']
Column 'Есть квартиры с черновой отделкой' has 3 unique values: ['да' 'неизвестно' 'нет']
Column 'Внутренний двор без машин' has 3 unique values: ['да' 'нет' 'неизвестно']
Column 'Видеонаблюдение во дворе' has 3 unique values: ['неизвестно' 'да' 'нет']
Column 'Закрытая территория' has 3 unique values: ['да' 'нет' 'неизвестно']


In [134]:
def preprocess_categorical_columns(ds, object_columns):
    for col in object_columns:
        if ds[col].dtype == 'object' and 'неизвестно' in ds[col].unique():
            target_yes = ds.loc[ds[col] == 'да', 'Таргет']
            target_no = ds.loc[ds[col] == 'нет', 'Таргет']
            median_yes = target_yes.median()
            median_no = target_no.median()
            unknown_indices = ds[ds[col] == 'неизвестно'].index

            for ind in unknown_indices:
                target_value = ds.loc[ind, 'Таргет']
                dist_yes = abs(target_value - median_yes)
                dist_no = abs(target_value - median_no)
                
                if dist_yes < dist_no:
                    ds.loc[ind, col] = 'да'
                else:
                    ds.loc[ind, col] = 'нет'
                    
    return ds


preprocess_categorical_columns(ds, object_list)

,Населенный пункт,Группа компаний,Класс недвижимости,Статус,Месяц,Таргет,Дата получения РС,Дата публикации ИНС,Всероссийский застройщик,Район,...,Расстояние по навигатору до: Луговая,Расстояние по навигатору до: Центр,Расстояние по навигатору до: Первая речка,Расстояние по навигатору до: Вторая речка,Расстояние по навигатору до: Третья рабочая,Расстояние по навигатору до: Окатовая,Расстояние по навигатору до: Баляева,Расстояние по навигатору до: Кампус ДВФУ,Расстояние по навигатору до: Артем,Расстояние по навигатору до: В-надеждинское
0,артем,сзвосход,типовой,строится,4,0.000000,19.08.2020,16.04.2021,нет,артем,...,3.510078,3.677804,3.473117,3.476863,3.673838,3.443989,3.585868,3.077331,-3.081237,-1.643038
1,артем,сзвосход,типовой,строится,5,0.000000,19.08.2020,16.04.2021,нет,артем,...,3.510078,3.677804,3.473117,3.476863,3.673838,3.443989,3.585868,3.077331,-3.081237,-1.643038
2,артем,сзвосход,типовой,строится,6,0.000000,19.08.2020,16.04.2021,нет,артем,...,3.510078,3.677804,3.473117,3.476863,3.673838,3.443989,3.585868,3.077331,-3.081237,-1.643038
3,артем,сзвосход,типовой,строится,7,0.000000,19.08.2020,16.04.2021,нет,артем,...,3.510078,3.677804,3.473117,3.476863,3.673838,3.443989,3.585868,3.077331,-3.081237,-1.643038
4,артем,сзвосход,типовой,строится,8,95000.000000,19.08.2020,16.04.2021,нет,артем,...,3.510078,3.677804,3.473117,3.476863,3.673838,3.443989,3.585868,3.077331,-3.081237,-1.643038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1602,вольно-надеждинское,сз днс дом,комфорт,строится,4,0.000000,22.06.2021,05.07.2021,нет,дальний пригород,...,1.863064,1.602181,1.750458,1.468385,1.669100,1.540538,1.608672,1.746956,-1.465829,-2.789875
1603,вольно-надеждинское,сз днс дом,комфорт,строится,5,0.000000,22.06.2021,05.07.2021,нет,дальний пригород,...,1.863064,1.602181,1.750458,1.468385,1.669100,1.540538,1.608672,1.746956,-1.465829,-2.789875
1604,вольно-надеждинское,сз днс дом,комфорт,строится,6,0.000000,22.06.2021,05.07.2021,нет,дальний пригород,...,1.863064,1.602181,1.750458,1.468385,1.669100,1.540538,1.608672,1.746956,-1.465829,-2.789875
1605,вольно-надеждинское,сз днс дом,комфорт,строится,7,0.000000,22.06.2021,05.07.2021,нет,дальний пригород,...,1.863064,1.602181,1.750458,1.468385,1.669100,1.540538,1.608672,1.746956,-1.465829,-2.789875


In [135]:
# checking unique values if there are more then 2

for col in object_list:
    num_unique = len(ds[col].unique())
    if num_unique != 2:
        print(f"Column '{col}' has {num_unique} unique values: {ds[col].unique()}")

In [136]:
def standardize_categorical_data(ds, object_columns):
    
    for col in object_columns:
        ds.loc[ds[col] == 'нет', col] = 0
        ds.loc[ds[col] == 'да', col] = 1
        ds[col] = ds[col].astype(int)
    
    return ds


In [137]:
standardize_categorical_data(ds, object_list)

,Населенный пункт,Группа компаний,Класс недвижимости,Статус,Месяц,Таргет,Дата получения РС,Дата публикации ИНС,Всероссийский застройщик,Район,...,Расстояние по навигатору до: Луговая,Расстояние по навигатору до: Центр,Расстояние по навигатору до: Первая речка,Расстояние по навигатору до: Вторая речка,Расстояние по навигатору до: Третья рабочая,Расстояние по навигатору до: Окатовая,Расстояние по навигатору до: Баляева,Расстояние по навигатору до: Кампус ДВФУ,Расстояние по навигатору до: Артем,Расстояние по навигатору до: В-надеждинское
0,артем,сзвосход,типовой,строится,4,0.000000,19.08.2020,16.04.2021,0,артем,...,3.510078,3.677804,3.473117,3.476863,3.673838,3.443989,3.585868,3.077331,-3.081237,-1.643038
1,артем,сзвосход,типовой,строится,5,0.000000,19.08.2020,16.04.2021,0,артем,...,3.510078,3.677804,3.473117,3.476863,3.673838,3.443989,3.585868,3.077331,-3.081237,-1.643038
2,артем,сзвосход,типовой,строится,6,0.000000,19.08.2020,16.04.2021,0,артем,...,3.510078,3.677804,3.473117,3.476863,3.673838,3.443989,3.585868,3.077331,-3.081237,-1.643038
3,артем,сзвосход,типовой,строится,7,0.000000,19.08.2020,16.04.2021,0,артем,...,3.510078,3.677804,3.473117,3.476863,3.673838,3.443989,3.585868,3.077331,-3.081237,-1.643038
4,артем,сзвосход,типовой,строится,8,95000.000000,19.08.2020,16.04.2021,0,артем,...,3.510078,3.677804,3.473117,3.476863,3.673838,3.443989,3.585868,3.077331,-3.081237,-1.643038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1602,вольно-надеждинское,сз днс дом,комфорт,строится,4,0.000000,22.06.2021,05.07.2021,0,дальний пригород,...,1.863064,1.602181,1.750458,1.468385,1.669100,1.540538,1.608672,1.746956,-1.465829,-2.789875
1603,вольно-надеждинское,сз днс дом,комфорт,строится,5,0.000000,22.06.2021,05.07.2021,0,дальний пригород,...,1.863064,1.602181,1.750458,1.468385,1.669100,1.540538,1.608672,1.746956,-1.465829,-2.789875
1604,вольно-надеждинское,сз днс дом,комфорт,строится,6,0.000000,22.06.2021,05.07.2021,0,дальний пригород,...,1.863064,1.602181,1.750458,1.468385,1.669100,1.540538,1.608672,1.746956,-1.465829,-2.789875
1605,вольно-надеждинское,сз днс дом,комфорт,строится,7,0.000000,22.06.2021,05.07.2021,0,дальний пригород,...,1.863064,1.602181,1.750458,1.468385,1.669100,1.540538,1.608672,1.746956,-1.465829,-2.789875


### object: multicategorical data 

In [138]:
object_ds = ds.select_dtypes(include=['object'])
object_columns = object_ds.columns

multi_categorical_cols = list(set(object_ds.columns.tolist()) - set(object_list))
object_ds[multi_categorical_cols].head()


,Основной материал фасада,Вспомогательный материал фасада,Группа компаний,Тип фасада,Тип дома,Дата сбора информации,Дата публикации ИНС,Класс недвижимости,Район,Месяц,Дата получения РС,Населенный пункт,Статус
0,камень,стекло,сзвосход,панель,монолит,22.12.2021,16.04.2021,типовой,артем,4,19.08.2020,артем,строится
1,камень,стекло,сзвосход,панель,монолит,22.12.2021,16.04.2021,типовой,артем,5,19.08.2020,артем,строится
2,камень,стекло,сзвосход,панель,монолит,22.12.2021,16.04.2021,типовой,артем,6,19.08.2020,артем,строится
3,камень,стекло,сзвосход,панель,монолит,22.12.2021,16.04.2021,типовой,артем,7,19.08.2020,артем,строится
4,камень,стекло,сзвосход,панель,монолит,22.12.2021,16.04.2021,типовой,артем,8,19.08.2020,артем,строится


In [139]:
def encode_categorical_features(ds, cols):

    ohe = OneHotEncoder()
    ohe.fit(ds[cols])
    ohe_data = ohe.transform(ds[cols]).toarray()
    ohe_cols = ohe.get_feature_names_out()
    ohe_df = pd.DataFrame(ohe_data, columns=ohe_cols)

    ds = pd.concat([ds.drop(cols, axis=1), ohe_df], axis=1)
    return ds

encoded_object_ds = encode_categorical_features(object_ds, multi_categorical_cols)
encoded_object_ds.head()


,Основной материал фасада_другой,Основной материал фасада_камень,Основной материал фасада_керамогранит,Основной материал фасада_кирпич,Основной материал фасада_металл,Основной материал фасада_пластик,Основной материал фасада_стекло,Вспомогательный материал фасада_дерево,Вспомогательный материал фасада_другой,Вспомогательный материал фасада_керамогранит,...,Дата получения РС_29.06.2018,Дата получения РС_29.12.2018,Дата получения РС_30.03.2021,Дата получения РС_30.08.2021,Дата получения РС_nan,Населенный пункт_артем,Населенный пункт_владивосток,Населенный пункт_вольно-надеждинское,Статус_сдан,Статус_строится
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [140]:
ds = pd.concat([ds.drop(object_ds.columns, axis=1), encoded_object_ds], axis=1)

ds.head()

,Таргет,Всероссийский застройщик,Инфраструктура: Рядом ВУЗ,Есть железная дорога 250м,Есть ЛЭП 100м,Есть магистральные дороги 100м,Есть АЗС 100м,Есть скверы 500м,Есть лес 500м,Есть водоем 500м,...,Дата получения РС_29.06.2018,Дата получения РС_29.12.2018,Дата получения РС_30.03.2021,Дата получения РС_30.08.2021,Дата получения РС_nan,Населенный пункт_артем,Населенный пункт_владивосток,Населенный пункт_вольно-надеждинское,Статус_сдан,Статус_строится
0,0.0,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.0,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.0,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,95000.0,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [141]:
ds.to_excel('src/dataset_processed.xlsx', index=False)